# pipeline - full recordings, resting state data

This notebook acts as the finalized pipeline for processing the full recordings of resting-state data, and computes PSDs, slopes, and outputs a samples-features matrix for statistical analysis. The analysis covers both eyes-closed and eyes-open data. Data has been preprocessed in MATLAB using the parameters and functions specified at `../data/pipeline-full/pipeline.txt`

The notebook is divided into the following sections:
0. **Parameters:** Parameter selection for running the analysis.
1. **PSD Calculations:** Loading subject PSDs for both older and younger adults.
2. **Fit to PSD Slopes:** Fitting to specified frequency range with specified fitting method (either simple linear regression or RANSAC).
3. **Construct Samples-Features Matrix:** Exporting results to a samples-features matrix, for statistical analysis.

In [1]:
%matplotlib inline
import os
import glob
import datetime
import seaborn as sns
import numpy as np
import scipy as sp
import pandas as pd
import scipy.io
import numpy.fft
import scipy.signal
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.stats import linregress, ttest_ind
from sklearn import linear_model
mpl.rcParams['figure.figsize'] = (16, 10)

## Parameters
- `recompute_psds`: `True` or `False`, for recomputing subject PSDs or loading previous results.
- `psd_buffer_lofreq`: Scalar, specifies the lower bound for the PSD buffer that we exclude.
- `psd_buffer_hifreq`: Scalar, specifies the upper bound for the PSD buffer that we exclude.
- `fitting_func`: `'linreg'` or `'ransac'`, specifies which function to use for fitting. `'linreg'` is simple linear regression. `'ransac'` is RANSAC, a robust fitting method that ignores outliers.
- `fitting_lofreq`: Scalar, specifies the lower bound for the PSD fitting range.
- `fitting_hifreq`: Scalar, specifies the upper bound for the PSD fitting range.
- `import_dir`: String specifying the directory to import results to.
- `export_dir`: String specifying the directory to export results to.

In [2]:
recompute_psds = True
psd_buffer_lofreq = 7
psd_buffer_hifreq = 14
fitting_func = 'ransac'
fitting_lofreq = 2
fitting_hifreq = 24
import_dir = '/Users/jorge/Drive/research/_psd-slope/data/rs-full/ExclFiltCARClust-mat/'
export_dir = '/Users/jorge/Drive/research/_psd-slope/data/rs-full/results/'

##### Set up workspace, print out parameters to text file...

In [3]:
current_time = '-'.join('_'.join(str(datetime.datetime.now()).split()).split(':'))[:-7]
export_dir = export_dir + current_time + '/'
os.mkdir(export_dir)
params = open(export_dir + 'parameters.txt', 'w')
params.write('recompute_psds = ' + str(recompute_psds))
params.write('\npsd_buffer_lofreq = ' + str(psd_buffer_lofreq))
params.write('\npsd_buffer_hifreq = ' + str(psd_buffer_hifreq))
params.write('\nfitting_func = ' + str(fitting_func))
params.write('\nfitting_lofreq = ' + str(fitting_lofreq))
params.write('\nfitting_hifreq = ' + str(fitting_hifreq))
params.write('\nimport_dir = ' + str(import_dir))
params.write('\nexport_dir = ' + str(export_dir))
params.close()

## Subject Importing & PSD Calculations

This section imports subject information and computes PSDs using Welch's method. The algorithm proceeds as follows, for each channel:
1. Extract as many clean 2-second eyes closed and eyes open segments from the recording. Segments overlap by 50%.
2. Multiply each 2-second segment by a 2-second Hamming window. 
3. Compute the discrete Fourier transform of each segment, and average DFT'd segments to arrive at a per-channel PSD.

The PSD is defined as:
$$
PSD = log_{10}(\sum\limits_{n=1}^{N}{N})
$$

##### Function Definitions

In [4]:
def get_filelist(import_path):
    matfiles = []
    for root, dirs, files in os.walk(import_path):
        matfiles += glob.glob(os.path.join(root, '*.mat'))
    return matfiles

def import_subject(subj, i, import_path):
    """ 
    Imports a single subject and adds them to the subj
    data structure. Additionally, merges 
    """
    subj[i] = {}
    datafile = sp.io.loadmat(import_path)
    subj[i]['name'] = str(np.squeeze(datafile['name']))
    subj[i]['srate'] = int(np.squeeze(datafile['srate']))
    subj[i]['events'] = []
    for event in np.squeeze(datafile['evts']):
        subj[i]['events'].append([event[0][0], event[1][0][0], event[2][0][0]])
    subj[i]['data'] = np.squeeze(datafile['data'])
    subj[i]['nbchan'] = len(subj[i]['data'])
    return subj

def _print_window_info(events, port_code):
    evts = [[events[i][1], events[i+1][1]] for i in range(len(events)) if events[i][0] == port_code]
    total_wins = 0
    total_secs = 0
    for e in evts:
        if (e[1] - e[0]) >= 1024:
            pts  = e[1] - e[0]
            secs = (e[1] - e[0])//512
            nwin = (e[1] - e[0])//512 - 1
            total_wins += nwin
            total_secs += secs
            print('Event {}:\t{} points, {} seconds, {} windows'.format(e, pts, secs, nwin))
    print('Total windows able to be extracted: ', total_wins)

def get_windows(data, events, port_code, nperwindow=512*2, noverlap=512):
    windows = []
    # The following line restructures events of type port_code into the 
    # following format:
    #         [start_time, end_time]
    evts = [[events[i][1], events[i+1][1]] for i in range(len(events)) if events[i][0] == port_code]
    for event in evts:
        if event[1]-event[0] >= nperwindow:
            nwindows = (event[1] - event[0])//noverlap - 1
            for i in range(nwindows):
                windows.append(data[event[0] + noverlap*i : event[0] + noverlap*i + nperwindow])
    return windows

def welch(windows, srate):
    """
    Takes a list of data segments (each size 1xN), computes each segment's PSD,
    and averages them to get a final PSD.
    """
    psds = [sp.signal.welch(window, srate, nperseg=len(window), window='hamming')[1] for window in windows]
    return np.mean(psds, axis=0)

def remove_freq_buffer(data, lofreq, hifreq):
    """
    Removes a frequency buffer from a PSD or frequency vector.
    """
    data = np.delete(data, range(lofreq*2, hifreq*2))
    return data.reshape(len(data), 1)

def compute_subject_psds(import_path, import_path_csv):
    """ Returns subj data structure with calculated PSDs and subject information.
    Arguments:
        import_path:     String, path to .mat files
        import_path_csv: String, path to .csv containing subject class, sex, and
                         age information. 
    """
    matfiles = get_filelist(import_path)
    df = pd.read_csv(import_path_csv)
    df.SUBJECT = df.SUBJECT.astype(str)

    subj = {}
    subj['nbsubj'] = len(matfiles)
    subj['f'] = np.linspace(0, 256, 513)
    subj['f'] = subj['f'].reshape(len(subj['f']), 1)
    subj['f_rm_alpha'] = remove_freq_buffer(subj['f'], 7, 14)
    for i in range(len(matfiles)):
        print("Processing: {}... ".format(matfiles[i].split('/')[-1]), end='')
        
        subj = import_subject(subj, i, matfiles[i])
        subj[i]['age']   = df[df.SUBJECT == subj[i]['name']].AGE.values[0]
        subj[i]['class'] = df[df.SUBJECT == subj[i]['name']].CLASS.values[0]
        subj[i]['sex']   = df[df.SUBJECT == subj[i]['name']].SEX.values[0]

        for ch in range(subj[i]['nbchan']):
            subj[i][ch] = {}
            eyesC_windows = get_windows(subj[i]['data'][ch], subj[i]['events'], 'C1')
            eyesO_windows = get_windows(subj[i]['data'][ch], subj[i]['events'], 'O1')
            subj[i][ch]['eyesC_psd'] = welch(eyesC_windows, 512)
            subj[i][ch]['eyesO_psd'] = welch(eyesO_windows, 512)
            subj[i][ch]['eyesC_psd_rm_alpha'] = remove_freq_buffer(subj[i][ch]['eyesC_psd'], 7, 14)
            subj[i][ch]['eyesO_psd_rm_alpha'] = remove_freq_buffer(subj[i][ch]['eyesO_psd'], 7, 14)
        subj[i]['data'] = np.nan # No longer needed, so clear it from memory
        subj[i]['eyesC_psd'] = np.mean([subj[i][ch]['eyesC_psd'] for ch in range(subj[i]['nbchan'])], axis=0)
        subj[i]['eyesO_psd'] = np.mean([subj[i][ch]['eyesO_psd'] for ch in range(subj[i]['nbchan'])], axis=0)
        subj[i]['eyesC_psd_rm_alpha'] = remove_freq_buffer(subj[i]['eyesC_psd'], 7, 14)
        subj[i]['eyesO_psd_rm_alpha'] = remove_freq_buffer(subj[i]['eyesO_psd'], 7, 14)
        print("Done.")
    return subj

##### Processing

In [5]:
if recompute_psds:
    # Import EEG for older and younger adults, compute PSDs
    subj = compute_subject_psds(import_dir, '../../data/ya-oa.csv')

    # Save resulting PSDs
    subj['time_computed'] = current_time
    np.save(export_dir + '/subj-no-fitting.npy', subj); subj = []
else:
    # Use files with pre-computed PSDs and a 7 - 14 Hz buffer
    !cp /Users/jorge/Dropbox/research/_psd-slope/data/rs-full/subj-no-fitting.npy $export_dir

Processing: 1121181181.mat... Done.
Processing: 1121181183.mat... Done.
Processing: 1121181218.mat... Done.
Processing: 1121181262.mat... Done.
Processing: 1121181286.mat... Done.
Processing: 112118131.mat... Done.
Processing: 1121181334.mat... Done.
Processing: 112118135.mat... Done.
Processing: 1121181393.mat... Done.
Processing: 1121181418.mat... Done.
Processing: 1121181424.mat... Done.
Processing: 1121181428.mat... Done.
Processing: 1121181510.mat... Done.
Processing: 1121181517.mat... Done.
Processing: 1121181575.mat... Done.
Processing: 112118167.mat... Done.
Processing: 112118204.mat... Done.
Processing: 112118257.mat... Done.
Processing: 112118266.mat... Done.
Processing: 112118334.mat... Done.
Processing: 112118373.mat... Done.
Processing: 112118416.mat... Done.
Processing: 112118463.mat... Done.
Processing: 112118468.mat... Done.
Processing: 112118475.mat... Done.
Processing: 112118479.mat... Done.
Processing: 112118521.mat... Done.
Processing: 112118526.mat... Done.
Process

## Fit to Spectral Slopes

Now we compute PSD slopes for each channel of each subject, and additionally calculate each subject's mean PSD slope. This is found by fitting to the grand average PSD of each subject.

##### Function Definitions

In [6]:
def linreg_slope(f, psd, lofreq, hifreq):
    """
    Fits line to the PSD, using simple linear regression.
    Returns slope and fit line.
    """
    model = linear_model.LinearRegression()
    model.fit(f[lofreq*2:hifreq*2], np.log10(psd[lofreq*2:hifreq*2]))
    fit_line = model.predict(f)
    return model.coef_[0] * (10**2), fit_line

def ransac_slope(f, psd, lofreq, hifreq):
    """
    Robustly fits line to the PSD, using the RANSAC algorithm. 
    Returns slope and fit line.
    """
    model_ransac = linear_model.RANSACRegressor(linear_model.LinearRegression())
    model_ransac.fit(f[lofreq*2:hifreq*2], np.log10(psd[lofreq*2:hifreq*2]))
    fit_line = model_ransac.predict(f)
    return model_ransac.estimator_.coef_[0] * (10**2), fit_line

def fit_slopes(subj, regr_func, lofreq, hifreq):
    """ 
    Takes subj data structure and fits slopes to each subject's PSDs and mean
    PSD, using regr_func and fitting to datapoints between lofreq and hifreq.
    """
    for i in range(subj['nbsubj']):
        # Per-subject PSD average fitting
        subj[i]['eyesC_slope'], subj[i]['eyesC_fitline'] = regr_func(subj['f'], subj[i]['eyesC_psd'], lofreq, hifreq)
        subj[i]['eyesO_slope'], subj[i]['eyesO_fitline'] = regr_func(subj['f'], subj[i]['eyesO_psd'], lofreq, hifreq)
        for ch in range(subj[i]['nbchan']):
            # Per-channel PSD fitting
            subj[i][ch]['eyesC_slope'], subj[i][ch]['eyesC_fitline'] = regr_func(subj['f'], subj[i][ch]['eyesC_psd_rm_alpha'], lofreq, hifreq)
            subj[i][ch]['eyesO_slope'], subj[i][ch]['eyesO_fitline'] = regr_func(subj['f'], subj[i][ch]['eyesO_psd_rm_alpha'], lofreq, hifreq)
    return subj

##### Processing

In [10]:
# Select fitting function
if fitting_func == 'linreg':
    regr = linreg_slope
elif fitting_func == 'ransac':
    regr = ransac_slope

#Load subject PSDs
if recompute_psds:
    subj = np.load(export_dir + '/subj-no-fitting.npy').item()
else:
    subj = np.load('../../data/rs-full/subj-no-fitting.npy').item()
    
# Fit lines to slopes using specified function and frequency range
subj = fit_slopes(subj, regr, fitting_lofreq, fitting_hifreq)

# Save results
filename = export_dir + 'subj-' + str(fitting_lofreq) + '-' + str(fitting_hifreq) + '-' + fitting_func + '.npy'
subj['time_computed'] = current_time
np.save(filename, subj); subj = []

/Users/jorge/Development/anaconda3/lib/python3.5/site-packages/scipy/linalg/basic.py:884: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


## Construct Samples-Features Matrix

Now we construct the samples-features matrix containing the calculated slopes. We use a table that already contains subject numbers, sex, age, and memory class to start off.

##### Function Definitions

In [11]:
def get_subject_slopes(subj, ch, slope_type):
    """ Returns list of slopes for specified channel of slope_type.
    Arguments:
        subj: The subj data structure.
        ch:   Scalar, channel for which to get list of subject slopes.
        slope_type: String, e.g., 'eyesO_slope' or 'eyesC_slope'
    """
    if ch == -1: # Slope of PSD grand average
        return [subj[i][slope_type]     for i in range(subj['nbsubj'])]
    else:
        return [subj[i][ch][slope_type][0] for i in range(subj['nbsubj'])]

##### Processing

In [12]:
# Define channels, these will form labels for our table:
channels = ["A01","A02","A03","A04","A05","A06","A07","A08","A09","A10","A11","A12","A13","A14","A15","A16","A17","A18","A19","A20","A21","A22","A23","A24","A25","A26","A27","A28","A29","A30","A31","A32","B01","B02","B03","B04","B05","B06","B07","B08","B09","B10","B11","B12","B13","B14","B15","B16","B17","B18","B19","B20","B21","B22","B23","B24","B25","B26","B27","B28","B29","B30","B31","B32","FRONTAL","LTEMPORAL","CENTRAL","RTEMPORAL","OCCIPITAL"]
# Load subject PSDs with fitted slopes
subj = np.load(filename).item()

data = {}
data['SUBJECT'] = [subj[i]['name'] for i in range(subj['nbsubj'])]
data['CLASS']   = [subj[i]['class'] for i in range(subj['nbsubj'])]
data['AGE']     = [subj[i]['age'] for i in range(subj['nbsubj'])]

df = pd.DataFrame(data)
df = df[['SUBJECT', 'CLASS', 'AGE']]

# Add each subject's mean slope.
df['AVG_PSD_EYESC'] = get_subject_slopes(subj, -1, 'eyesC_slope')
df['AVG_PSD_EYESO'] = get_subject_slopes(subj, -1, 'eyesO_slope')

# Now add slopes for every channel from each subject.
for ch in range(len(channels)):
    df[channels[ch] + '_EYESC'] = get_subject_slopes(subj, ch, 'eyesC_slope')
for ch in range(len(channels)):
    df[channels[ch] + '_EYESO'] = get_subject_slopes(subj, ch, 'eyesO_slope')

# Export results
filename = export_dir + 'ya-oa-full-' + fitting_func + '-' + str(fitting_lofreq) + '-' + str(fitting_hifreq) + '-eyesc-eyeso.csv'
print('Saving fitted slopes at:\n', filename)
df.to_csv(filename, index=False); df = []

Saving fitted slopes at:
 /Users/jorge/Drive/research/_psd-slope/data/rs-full/results/2016-11-15_15-19-23/ya-oa-full-ransac-2-24-eyesc-eyeso.csv


## Compute t-tests, logit, lasso

And run t-tests, as well as LASSO in order to see if there are any group differences. 

### T-Tests

##### Younger adults vs older adults

In [13]:
df = pd.read_csv(filename)
ya = df[df.CLASS.isin(['DANE'])]
oa = df[df.CLASS.isin(['SA_Control', 'MCI_Control'])]

channels = list(df.columns.values)[4:]
for ch in channels:
    result = ttest_ind(ya[ch], oa[ch], equal_var=False)
    if result[1] < 0.05:
        print("{}:\t\t\t{:.2f},\t{:.3f}".format(ch, result.statistic, result.pvalue))

A28_EYESC:			-2.53,	0.014
A32_EYESC:			-2.82,	0.007
B17_EYESC:			-2.01,	0.049
B25_EYESC:			-2.25,	0.029
B28_EYESC:			2.01,	0.049
B31_EYESC:			-2.09,	0.041
B32_EYESC:			-2.42,	0.019
A28_EYESO:			-2.02,	0.048
A32_EYESO:			-2.08,	0.042
B25_EYESO:			-2.01,	0.049


##### Older adult controls vs SAs

In [14]:
oa_control = df[df.CLASS.isin(['SA_Control', 'MCI_Control'])]
sa         = df[df.CLASS.isin(['SA'])]

for ch in channels:
    result = ttest_ind(oa_control[ch], sa[ch], equal_var=False)
    if result[1] < 0.05:
        print("{}:\t\t\t{:.2f},\t{:.3f}".format(ch, result.statistic, result.pvalue))

B12_EYESC:			-2.11,	0.047
B24_EYESC:			-2.04,	0.050
B27_EYESC:			-2.25,	0.034
A05_EYESO:			-2.14,	0.039
A07_EYESO:			-2.07,	0.047
A13_EYESO:			-2.71,	0.011
A15_EYESO:			-2.22,	0.035
A19_EYESO:			-2.14,	0.039
B01_EYESO:			-2.13,	0.042
B02_EYESO:			-2.96,	0.006
B03_EYESO:			-3.03,	0.004
B05_EYESO:			-2.45,	0.020
B06_EYESO:			-3.20,	0.003
B20_EYESO:			-2.31,	0.029
B22_EYESO:			-2.16,	0.037
B23_EYESO:			-2.57,	0.015
B24_EYESO:			-2.22,	0.032
B27_EYESO:			-2.43,	0.023
B29_EYESO:			-2.28,	0.029
FRONTAL_EYESO:			-2.73,	0.009
CENTRAL_EYESO:			-2.29,	0.030
RTEMPORAL_EYESO:			-2.24,	0.031
OCCIPITAL_EYESO:			-2.05,	0.048


### Logistic Regression, younger adults vs older adult controls

In [15]:
df_logit = df[df.CLASS.isin(['DANE', 'SA_Control', 'MCI_Control'])]

df_logit['CLASS'] = list(map(lambda x: 0 if x == 'DANE' else 1, df_logit['CLASS']))

cols = list(df_logit.columns.values)
cols.remove('SUBJECT')
cols.remove('CLASS')
cols.remove('AGE')

X = df_logit[cols]
y = df_logit.CLASS

/Users/jorge/Development/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [16]:
import warnings                 # sklearn is using a deprecated rand function here,
with warnings.catch_warnings(): # and warnings clutter output
    warnings.simplefilter("ignore")
    resamplings = 2000
    rlogit = linear_model.RandomizedLogisticRegression(n_resampling=resamplings)
    rlogit.fit(X, y)
    print("Features sorted by score, using {} resamplings: ".format(resamplings))
    feature_list = sorted(zip(map(lambda x: round(x, 4), rlogit.scores_), cols), reverse=True)
    for f in feature_list[0:25]: # Adjust this if last feature output is nonzero
        print("{}:\t\t\t{:.2f}".format(f[1], f[0]))

Features sorted by score, using 2000 resamplings: 
A32_EYESC:			0.26
A28_EYESC:			0.11
B32_EYESC:			0.08
B25_EYESC:			0.07
B28_EYESC:			0.05
B17_EYESC:			0.03
B31_EYESC:			0.03
A32_EYESO:			0.03
B12_EYESC:			0.02
B22_EYESC:			0.02
B25_EYESO:			0.02
A28_EYESO:			0.02
A25_EYESC:			0.01
AVG_PSD_EYESO:			0.01
A01_EYESC:			0.01
B32_EYESO:			0.01
A20_EYESC:			0.00
A13_EYESO:			0.00
B22_EYESO:			0.00
A20_EYESO:			0.00
RTEMPORAL_EYESC:			0.00
B30_EYESC:			0.00
B12_EYESO:			0.00
A10_EYESC:			0.00
B26_EYESC:			0.00


### Entire dataset, LASSO for age as interest variable.

In [17]:
X, y = df[cols], df.AGE

import warnings                 # sklearn is using a deprecated rand function here,
with warnings.catch_warnings(): # and warnings clutter output
    warnings.simplefilter("ignore")
    resamplings = 2000
    rlasso = linear_model.RandomizedLasso(n_resampling=resamplings)
    rlasso.fit(X, y)
    print("Features sorted by score, using {} resamplings: ".format(resamplings))
    feature_list = sorted(zip(map(lambda x: round(x, 4), rlasso.scores_), cols), reverse=True)
    for f in feature_list[0:50]: # Adjust this if last feature output is nonzero
        print("{}:\t\t\t{:.2f}".format(f[1], f[0]))

Features sorted by score, using 2000 resamplings: 
A32_EYESC:			0.45
B32_EYESC:			0.29
B25_EYESO:			0.24
A28_EYESO:			0.23
B25_EYESC:			0.17
B17_EYESC:			0.08
A28_EYESC:			0.06
B32_EYESO:			0.03
B17_EYESO:			0.03
AVG_PSD_EYESO:			0.03
A32_EYESO:			0.02
A21_EYESO:			0.02
A27_EYESC:			0.01
B15_EYESC:			0.01
A20_EYESO:			0.01
A20_EYESC:			0.01
B01_EYESO:			0.01
A10_EYESC:			0.01
RTEMPORAL_EYESO:			0.01
B02_EYESO:			0.01
RTEMPORAL_EYESC:			0.01
A01_EYESC:			0.01
B31_EYESC:			0.01
B22_EYESC:			0.00
B30_EYESC:			0.00
B16_EYESO:			0.00
A16_EYESC:			0.00
B19_EYESC:			0.00
B09_EYESO:			0.00
A08_EYESO:			0.00
LTEMPORAL_EYESC:			0.00
B28_EYESC:			0.00
B16_EYESC:			0.00
B03_EYESO:			0.00
A12_EYESO:			0.00
B24_EYESC:			0.00
A23_EYESC:			0.00
A16_EYESO:			0.00
LTEMPORAL_EYESO:			0.00
FRONTAL_EYESO:			0.00
B31_EYESO:			0.00
B24_EYESO:			0.00
B22_EYESO:			0.00
B15_EYESO:			0.00
B11_EYESC:			0.00
B09_EYESC:			0.00
B07_EYESC:			0.00
A25_EYESC:			0.00
A24_EYESO:			0.00
A22_EYESO:			0.00
